### Продвинутый SQL. Часть 1. Задачи.
- Входные данные: базой данных StackOverflow — сервиса вопросов и ответов о программировании.

In [2]:
from IPython.display import Image
from IPython.core.display import HTML
Image(url= "https://pictures.s3.yandex.net/resources/Frame_353_1_1665399433.png")

1.
Найдите количество вопросов, которые набрали больше 300 очков или как минимум 100 раз были добавлены в «Закладки».

In [ ]:
SELECT COUNT(posts.id)
FROM stackoverflow.posts posts
JOIN stackoverflow.post_types pt ON posts.post_type_id=pt.id
WHERE type = 'Question'
AND score >300 OR favorites_count >=100;

2.
Сколько в среднем в день задавали вопросов с 1 по 18 ноября 2008 включительно? Результат округлите до целого числа.

In [ ]:
WITH s1 AS (SELECT COUNT (DISTINCT posts.id) as question, creation_date::date
FROM stackoverflow.posts posts
JOIN stackoverflow.post_types pt ON posts.post_type_id=pt.id
WHERE type = 'Question'
AND creation_date::date BETWEEN '01-11-2008' AND '18-11-2008'
GROUP BY 2)
SELECT ROUND(AVG(question))
FROM s1;

3.
Сколько пользователей получили значки сразу в день регистрации? Выведите количество уникальных пользователей.

In [ ]:
SELECT COUNT (DISTINCT u.id)
FROM stackoverflow.users u
JOIN stackoverflow.badges b ON u.id=b.user_id
WHERE u.creation_date::date = b.creation_date::date;

4.
Сколько уникальных постов пользователя с именем Joel Coehoorn получили хотя бы один голос?

In [ ]:
SELECT COUNT(DISTINCT p.id)
FROM stackoverflow.users u
JOIN stackoverflow.posts p ON u.id = p.user_id
JOIN stackoverflow.votes v ON p.id=v.post_id
WHERE display_name = 'Joel Coehoorn'
;

5.
Выгрузите все поля таблицы vote_types. Добавьте к таблице поле rank, в которое войдут номера записей в обратном порядке. Таблица должна быть отсортирована по полю id.

In [ ]:
SELECT *, ROW_NUMBER() OVER (ORDER BY id DESC)
FROM stackoverflow.vote_types
ORDER BY id;

6.
Отберите 10 пользователей, которые поставили больше всего голосов типа Close. Отобразите таблицу из двух полей: идентификатором пользователя и количеством голосов. 

Отсортируйте данные сначала по убыванию количества голосов, потом по убыванию значения идентификатора пользователя.

In [ ]:
SELECT DISTINCT(u.id), COUNT(vt.id)
FROM stackoverflow.users u
JOIN stackoverflow.votes v ON u.id=v.user_id
JOIN stackoverflow.vote_types vt ON v.vote_type_id = vt.id
WHERE vt.name = 'Close'
GROUP BY 1
ORDER BY COUNT(vt.id) DESC, id DESC
LIMIT 10;

7.
Отберите 10 пользователей по количеству значков, полученных в период с 15 ноября по 15 декабря 2008 года включительно.

Отобразите несколько полей:
- идентификатор пользователя;
- число значков;
- место в рейтинге — чем больше значков, тем выше рейтинг.

Пользователям, которые набрали одинаковое количество значков, присвойте одно и то же место в рейтинге.
Отсортируйте записи по количеству значков по убыванию, а затем по возрастанию значения идентификатора пользователя.

In [ ]:
SELECT id, total,
       DENSE_RANK() OVER (ORDER BY total DESC) as rank
FROM (SELECT DISTINCT u.id, COUNT(b.id) as total
FROM stackoverflow.badges b
JOIN stackoverflow.users u ON b.user_id = u.id
WHERE b.creation_date BETWEEN '15-11-2008' AND '16-12-2008'
GROUP BY u.id
ORDER BY total DESC) as s1
LIMIT 10;

8.
Сколько в среднем очков получает пост каждого пользователя?
Сформируйте таблицу из следующих полей:
- заголовок поста;
- идентификатор пользователя;
- число очков поста;
- среднее число очков пользователя за пост, округлённое до целого числа.

Не учитывайте посты без заголовка, а также те, что набрали ноль очков.

In [ ]:
WITH s1 AS (SELECT p.id,p.title, p.user_id, p.score
FROM stackoverflow.posts p
WHERE title IS NOT NULL
AND score <> 0)
SELECT title, user_id, score,
ROUND(AVG(score) OVER (PARTITION BY user_id))
FROM s1;

9.
Отобразите заголовки постов, которые были написаны пользователями, получившими более 1000 значков. Посты без заголовков не должны попасть в список.

In [ ]:
SELECT title
FROM (SELECT user_id, achi
FROM (SELECT DISTINCT b.user_id, COUNT(id) as achi
FROM stackoverflow.badges b
GROUP BY 1) as s1
WHERE achi > 1000) as s2
JOIN stackoverflow.users u ON s2.user_id = u.id
JOIN stackoverflow.posts p ON u.id = p.user_id
WHERE title IS NOT NULL
;

10.
Напишите запрос, который выгрузит данные о пользователях из США (англ. United States). 

Разделите пользователей на три группы в зависимости от количества просмотров их профилей:
- пользователям с числом просмотров больше либо равным 350 присвойте группу 1;
- пользователям с числом просмотров меньше 350, но больше либо равно 100 — группу 2;
- пользователям с числом просмотров меньше 100 — группу 3.

Отобразите в итоговой таблице идентификатор пользователя, количество просмотров профиля и группу. Пользователи с нулевым количеством просмотров не должны войти в итоговую таблицу.

In [ ]:
WITH s1 AS (SELECT *
FROM stackoverflow.users
WHERE location LIKE '%United States%')
SELECT id, views,
CASE
           WHEN views < 100 THEN '3'
           WHEN views >= 350 THEN '1'
           ELSE 2
END AS category
FROM s1
WHERE views <> 0;

11.
Дополните предыдущий запрос. Отобразите лидеров каждой группы — пользователей, которые набрали максимальное число просмотров в своей группе. 

Выведите поля с идентификатором пользователя, группой и количеством просмотров. Отсортируйте таблицу по убыванию просмотров, а затем по возрастанию значения идентификатора.

In [ ]:
WITH tab AS 
(SELECT t.id, t.views, t.group, 
MAX(t.views) OVER (PARTITION BY t.group) AS max 
FROM 
(SELECT id, views, 
CASE 
WHEN views>=350 THEN 1 
WHEN views<100 THEN 3 
ELSE 2 
END AS group 
FROM stackoverflow.users 
WHERE location LIKE '%United States%' 
AND views != 0) as t) 
 
SELECT tab.id, tab.views, tab.group 
FROM tab 
WHERE tab.views = tab.max 
ORDER BY tab.views DESC, tab.id;

12.
Посчитайте ежедневный прирост новых пользователей в ноябре 2008 года. Сформируйте таблицу с полями:
- номер дня;
- число пользователей, зарегистрированных в этот день;
- сумму пользователей с накоплением.

In [ ]:
SELECT days, per_day, SUM(per_day) OVER (ORDER BY days)
FROM (SELECT DISTINCT EXTRACT(DAY FROM creation_date) as days, COUNT(DISTINCT id) as per_day
FROM stackoverflow.users
WHERE creation_date BETWEEN '1-11-2008' AND '1-12-2008'
GROUP BY EXTRACT(DAY FROM creation_date)) k
ORDER BY days

13.
Для каждого пользователя, который написал хотя бы один пост, найдите интервал между регистрацией и временем создания первого поста. 

Отобразите:
- идентификатор пользователя;
- разницу во времени между регистрацией и первым постом.

In [ ]:
SELECT DISTINCT p.user_id, MIN(p.creation_date) OVER (PARTITION BY p.user_id) - u.creation_date
FROM stackoverflow.posts p
JOIN stackoverflow.users u ON u.id = p.user_id